# Books recommendation system using clustering | COLLABORATIVE BASED

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\arbru\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
books = pd.read_csv('BX-Books.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
books.head()

C:\Users\arbru\AppData\Local\Temp\ipykernel_19140\1042601973.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
b_shape = books.shape

## EDA

In [4]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [5]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [6]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "publication",
    "Publisher": "publisher",
    "Image-URL-L":"img_url",
    
}, inplace = True)

In [7]:
books.head(2)

,ISBN,title,author,publication,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [8]:
# importing the users data set
users = pd.read_csv("BX-Users.csv", sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [9]:
u_shape=users.shape

In [10]:
# importing the ratings data set
ratings = pd.read_csv("BX-Book-Ratings.csv", sep = ";", on_bad_lines = 'skip', encoding = 'latin-1')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [11]:
r_shape = ratings.shape

In [12]:
print(b_shape)
print(u_shape)
print(r_shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [13]:
ratings.rename(columns={
    "User-ID":"user_id",
    "Book-Rating":"rating",
}, inplace=True)
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [14]:
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user_id, Length: 105283, dtype: int64

In [15]:
f"There are {ratings['user_id'].unique().shape} unique users"

'There are (105283,) unique users'

In [16]:
# Who has rated more than 200 books?
x = ratings['user_id'].value_counts() > 200
x[x].shape

(899,)

In [17]:
y = x[x].index
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [18]:
# Using the y index, let's find out the user ids.

ratings = ratings[ratings['user_id'].isin(y)]
# So, these are the users that have rated more than 200 books
ratings

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [19]:
ratings.shape

(526356, 3)

In [20]:
ratings_w_books = ratings.merge(books, on = "ISBN")
ratings_w_books

,user_id,ISBN,rating,title,author,publication,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,http://images.amazon.com/images/P/1892145022.0...
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...


In [21]:
ratings_w_books.shape

(487671, 8)

In [22]:
num_rating = ratings_w_books.groupby('title')['rating'].count().reset_index()
num_rating

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
160264,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
160265,Ã?Â?lpiraten.,1
160266,Ã?Â?rger mit Produkt X. Roman.,1
160267,Ã?Â?stlich der Berge.,1


In [23]:
num_rating.rename(columns={"rating":"num_of_rating"}, inplace=True)
num_rating.head(2)

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [24]:
final_rating = ratings_w_books.merge(num_rating, on = "title")
final_rating.head(2)

,user_id,ISBN,rating,title,author,publication,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [25]:
final_rating.shape

(487671, 9)

In [26]:
final_rating = final_rating[final_rating['num_of_rating']>=50]
final_rating.head()

,user_id,ISBN,rating,title,author,publication,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [27]:
final_rating.shape

(61853, 9)

In [28]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,publication,publisher,img_url,num_of_rating
169684,35857,0671741195,0,The Cradle Will Fall,Mary Higgins Clark,1991,Pocket,http://images.amazon.com/images/P/0671741195.0...,58
57618,87555,0743225422,0,The Shipping News : A Novel,Annie Proulx,2001,Scribner,http://images.amazon.com/images/P/0743225422.0...,71
32216,12538,0425140032,0,Dragon Tears,Dean R. Koontz,1995,Berkley Publishing Group,http://images.amazon.com/images/P/0425140032.0...,54
158862,200226,0345439104,0,Drowning Ruth (Oprah's Book Club),CHRISTINA SCHWARZ,2001,Ballantine Books,http://images.amazon.com/images/P/0345439104.0...,63
119792,174304,0380778556,0,Rebecca,Daphne Du Maurier,1994,Avon,http://images.amazon.com/images/P/0380778556.0...,68
15640,102647,0440234743,0,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,182
129415,224764,0821776037,0,Whispers,Lisa Jackson,2003,Kensington Publishing Corporation,http://images.amazon.com/images/P/0821776037.0...,129
48877,170518,0688042171,0,If Tomorrow Comes,Sidney Sheldon,1985,William Morrow &amp; Company,http://images.amazon.com/images/P/0688042171.0...,59
82332,159033,059035342X,0,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books,http://images.amazon.com/images/P/059035342X.0...,183
35396,105979,0440226430,0,Summer Sisters,Judy Blume,1999,Dell Publishing Company,http://images.amazon.com/images/P/0440226430.0...,188


In [29]:
#Checking for null values
final_rating.isnull().sum()

user_id          0
ISBN             0
rating           0
title            0
author           0
publication      0
publisher        0
img_url          0
num_of_rating    0
dtype: int64

In [30]:
# Dropping duplicates
final_rating.drop_duplicates(['title', 'user_id'], inplace=True)

In [31]:
final_rating.shape

(59850, 9)

In [32]:
final_rating

,user_id,ISBN,rating,title,author,publication,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
...,...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,50


In [33]:
final_rating.groupby('author')['rating'].count()

author
A. Manette Ansay    104
A.S. BYATT           56
ANN BRASHARES        50
ANNA QUINDLEN       157
ANNE FRANK           62
                   ... 
Winston Groom        70
Yann Martel         179
Yann Queffelec        1
Zachary Alan Fox     10
Zadie Smith          52
Name: rating, Length: 579, dtype: int64

# Pivot Table

In [34]:
# Generates a matrix that will contain the user_id, book_title, and ratings

# user_id| 255.. |255..|
# --------------------------- cluster the similarities based on what user one has done or smth like that
# title  |       |     |
# ---------------------------
# HP     |9      |  3  |
# ---------------------------
# 4blondes|4     |   10|
# ---------------------------

In [35]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [36]:
book_pivot.shape

(742, 888)

In [37]:
book_pivot.fillna(0, inplace=True)

In [38]:
book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [39]:
from scipy.sparse import csr_matrix

In [40]:
book_sparse = csr_matrix(book_pivot) #compresses the matrix
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

# Model Building

In [41]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(algorithm='brute')

In [42]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
# let's pass 237 -> harry potter id and get the suggestions
# n_neighbors used to determine how many book suggestions you'll get
# .reshape(1,-1) -> 1-dimensional array to 2 dimensional array with 1 row and as many columns as needed.
# values -> converts the array into a numpy array
# distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors = n)

In [48]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1), n_neighbors = 6)

In [49]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [50]:
di, sug = model.kneighbors(book_pivot.iloc[422, :].values.reshape(1,-1), n_neighbors = 6)

In [51]:
for j in range(len(sug)):
    print(book_pivot.index[sug[j]])

Index(['River's End', 'Homeport', 'The Reef', 'Dark Angel',
       'Truly, Madly Manhattan', 'Sanctuary'],
      dtype='object', name='title')


In [ ]:
distance

In [ ]:
suggestion

In [ ]:
book_pivot.index[2]

In [ ]:
book_pivot.index

In [ ]:
books_name = book_pivot.index

In [ ]:
import pickle
pickle.dump(model, open('artifacts/model.pkl','wb')) # dumping the model as a pickle file
pickle.dump(books_name, open('artifacts/books_name.pkl','wb')) # dumping the books_name(book_pivot.index) into a pickle file
pickle.dump(final_rating, open('artifacts/final_rating.pkl','wb')) # dumping the final_rating into a pickle file
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

In [60]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors=6)
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            if j != book_name:
                print(j)

In [61]:
book_name ='A Bend in the Road'
recommend_book(book_name)

Exclusive
The Cradle Will Fall
No Safe Place
Family Album
Lake Wobegon days
